# Getting Data

This notebook demonstrates a few basic examples from matminer's data retrieval features.  Matminer supports data retrival from the following sources.

* [Materials Project](https://materialsproject.org)
* [Citrine Informatics](https://citrination.com)
* [The Materials Platform for Data Science (MPDS)](mpds.io)
* [The Materials Data Facility](https://materialsdatafacility.org/)

Each resource has a corresponding object in matminer designed for retrieving data and preprocessing it into a pandas dataframe.  In addition, matminer can also access and aggregate data from your own [mongo database](https://www.mongodb.com/), if you have one.

## Materials Project

The materials project data retrieval tool, `matminer.data_retrieval.retrieve_MP.MPDataRetrieval` is initialized using an api_key that can be found on your personal dashboard page on [materialsproject.org](materialsproject.org) if you've created an account.  If you've set your api key via pymatgen (e.g. `pmg config --add PMG_MAPI_KEY YOUR_API_KEY_HERE`), the data retrieval tool may be initialized without an input argument.

In [1]:
from matminer.data_retrieval.retrieve_MP import MPDataRetrieval

In [2]:
mpdr = MPDataRetrieval() # or MPDataRetrieval(api_key=YOUR_API_KEY here)

Getting a dataframe corresponding to the materials project is essentially equivalent to using the MPRester's query method.(see [`pymatgen.ext.matproj.MPRester`](http://pymatgen.org/_modules/pymatgen/ext/matproj.html))  The inputs are  `criteria`, a mongo-style dictionary with which to filter the data, and `properties`, a list of supported properties which to return.  See the [MAPI documentation](https://github.com/materialsproject/mapidoc/tree/master/materials) for a list of and information about supported properties.

#### Example 1: Get densities of all elemental materials, i. e. those that contain one element

In [3]:
df = mpdr.get_dataframe({"nelements": 1}, ['density', 'pretty_formula'])
print("There are {} entries on MP with 1 element".format(df['density'].count()))

There are 466 entries on MP with 1 element


In [4]:
df.head()

,density,pretty_formula
material_id,,
mp-862690,8.281682,Ac
mp-10018,8.305508,Ac
mp-124,9.948341,Ag
mp-989737,9.922632,Ag
mp-8566,9.909384,Ag


#### Example 2: Get all bandgaps larger than 3.0 eV

In [5]:
df = mpdr.get_dataframe({"band_gap": {"$gt": 4.0}}, ['band_gap'])

In [6]:
print("There are {} entries on MP with a band gap larger than 4.0".format(df['band_gap'].count()))

There are 5841 entries on MP with a band gap larger than 4.0


In [7]:
df.head()

,band_gap
material_id,
mp-558642,4.0001
mp-558629,4.0007
mp-540140,4.0007
mp-780534,4.0007
mp-634434,4.0009


#### Example 3: Get all VRH shear and bulk moduli from the "elasticity" sub-document for which no warnings are found

In [8]:
df = mpdr.get_dataframe({"elasticity": {"$exists": True}, "elasticity.warnings": None},
                        ['elasticity.K_VRH', 'elasticity.G_VRH'])

In [9]:
print("There are {} elastic entries on MP with no warnings".format(df['elasticity.K_VRH'].count()))

There are 5790 elastic entries on MP with no warnings


In [10]:
df.head()

,elasticity.K_VRH,elasticity.G_VRH
material_id,,
mvc-11484,172.0,101.0
mvc-11438,37.0,14.0
mvc-15350,107.0,45.0
mvc-16396,38.0,23.0
mvc-652,83.0,44.0


## Citrine informatics

The materials project data retrieval tool, `matminer.data_retrieval.retrieve_Citrine.CitrineDataRetrieval` is initialized using an api_key that can be found on your "Account Settings" tab under your username in the upper right hand corner of the user interface at [citrination.com](citrination.com).  You can also set an environment variable, `CITRINE_KEY` to have your API key read automatically by the citrine informatics python API, (e. g. put `export CITRINE_KEY=YOUR_API_KEY_HERE` into your .bashrc).

In [11]:
from matminer.data_retrieval.retrieve_Citrine import CitrineDataRetrieval

#### Example 1: Get band gaps of various entries with formula PbTe

In [12]:
cdr = CitrineDataRetrieval() # or CitrineDataRetrieval(api_key=YOUR_API_KEY) if $CITRINE_KEY is not set

In [13]:
df = cdr.get_dataframe(formula='Si', prop='band gap', data_type='EXPERIMENTAL')

100%|██████████| 7/7 [00:00<00:00, 101.58it/s]


In [14]:
df.head()

,chemicalFormula,references,Band gap,Band gap-conditions,Band gap-dataType,Band gap-methods,Band gap-units,Crystallinity
system,,,,,,,,
1,Si,[{'doi': '10.1063/1.3253115'}],1.12,"[{'name': 'Transition', 'scalars': [{'value': ...",EXPERIMENTAL,[{'name': 'Absorption'}],eV,Single crystalline
2,Si,[{'doi': '10.1063/1.3253115'}],1.166,"[{'name': 'Transition', 'scalars': [{'value': ...",EXPERIMENTAL,NaN,eV,Single crystalline
3,Si,[{'doi': '10.1063/1.3253115'}],1.16,"[{'name': 'Transition', 'scalars': [{'value': ...",EXPERIMENTAL,[{'name': 'Absorption'}],eV,Single crystalline
4,Si,[{'doi': '10.1063/1.3253115'}],1.16,"[{'name': 'Transition', 'scalars': [{'value': ...",EXPERIMENTAL,[{'name': 'Absorption'}],eV,Single crystalline
5,Si,[{'doi': '10.1063/1.3253115'}],1.17,"[{'name': 'Transition', 'scalars': [{'value': ...",EXPERIMENTAL,[{'name': 'Absorption'}],eV,Single crystalline


#### Example 2: Get adsorption energies of O\* and OH\*

In [15]:
df_OH = cdr.get_dataframe(prop='adsorption energy of OH')
df_O = cdr.get_dataframe(prop='adsorption energy of O')

100%|██████████| 21/21 [00:00<00:00, 119.29it/s]


In [16]:
df_OH.head()

,chemicalFormula,references,Adsorption energy of OH,Adsorption energy of OH-conditions,Adsorption energy of OH-dataType,Adsorption energy of OH-units,Morphology,Surface facet
system,,,,,,,,
1,Pt,"[{'citation': '10.1039/c2cc30281k', 'doi': '10...",2.44,NaN,NaN,eV,NaN,(111)
2,Cu,"[{'citation': '10.1016/s1872-2067(12)60642-1',...",-3.55,NaN,COMPUTATIONAL,eV,NaN,(211)
3,ZnO,"[{'citation': '10.1016/s1872-2067(12)60642-1',...",-3.03,NaN,COMPUTATIONAL,eV,Thin film,NaN
4,Fe,"[{'citation': '10.1016/j.corsci.2012.11.011', ...",-3.95,NaN,NaN,eV,NaN,(100)
5,Pt,"[{'citation': '10.1021/jp807094m', 'doi': '10....",2.71,"[{'name': 'Site', 'scalars': [{'value': 'Top s...",NaN,eV,NaN,(111)


In [17]:
df_O.head()

,chemicalFormula,references,Adsorption energy of O,Adsorption energy of O-conditions,Adsorption energy of O-units,Reconstruction,Surface facet
system,,,,,,,
1,Fe,"[{'citation': '10.1016/j.jcat.2007.04.018', 'd...",-5.42,NaN,eV,NaN,(111)
2,Pt,"[{'citation': '10.1002/cctc.201100308', 'doi':...",1.53,NaN,eV,NaN,(111)
3,Pt,"[{'citation': '10.1021/jp307055j', 'doi': '10....",-4.54,NaN,eV,NaN,(111)
4,Co,"[{'citation': '10.1021/jp710674q', 'doi': '10....",2.37,"[{'name': 'Site', 'scalars': [{'value': 'FCC s...",eV,NaN,(0001)
5,Rh,"[{'citation': '10.1007/bf00806980', 'doi': '10...",-300,NaN,kJ/mol,NaN,(110)


## MPDS - The Materials Platform for Data Science

The [Materials Platform for Data Science](https://mpds.io/) interface is contained in `matminer.data_retrieval.retrieve_MPDS.MPDSDataRetrieval`, and is invoked using an API key and an optional endpoint.  Similarly to the Citrine and MP interfaces, MPDS can be invoked without specifying your API key if MPDS_KEY is set as an environment variable (e. g. put `export MPDS_KEY=YOUR_MPDS_KEY` into your .bashrc or .bash_profile).

In [18]:
from matminer.data_retrieval.retrieve_MPDS import MPDSDataRetrieval

In [19]:
mpdsdr = MPDSDataRetrieval() # or MPDSDataRetrieval(api_key=YOUR_API_KEY)

The `get_dataframe` method of the MPDSDataRetrieval class uses a search functionality documented on the [MPDS website](http://developer.mpds.io/#Categories).  Basically, the `search` keyword argument should take a dictionary with keys and values corresponding to search categories and values.  Note that the search functionality of the MPDS interface may be severely limited without full (i.e. paid subscription) access to the database.

In [20]:
df = mpdsdr.get_dataframe(search={"elements": "K-Ag", 
                                  "props": "heat capacity"})


Got 5 hits


In [21]:
df.head()

,Phase,Formula,SG,Entry,Property,Units,Value
0,30000,KAg4I5 rt,213,P1201629-3,heat capacity at constant pressure,J K-1 g-at.-1,30.5
1,79286,K2NaAg3[CN]6 lt,12,P1307433-3,heat capacity at constant pressure,J K-1 g-at.-1,83.0
2,79286,K2NaAg3[CN]6 lt,12,P1307433-4,heat capacity at constant pressure,J K-1 g-at.-1,112.0
3,79286,K2NaAg3[CN]6 lt,12,P1307434-3,heat capacity at constant pressure,J K-1 g-at.-1,78.0
4,79286,K2NaAg3[CN]6 lt,12,P1307434-4,heat capacity at constant pressure,J K-1 g-at.-1,11.0


## MDF - The Materials Data Facility


The MDF data retrieval tool, `matminer.data_retrieval.retrieve_MDF.MDFDataRetrieval` is initialized using a globus initialization key.  Upon the first invocation of a MDFDataRetrieval object, you should be prompted with a string of numbers and letters you can enter on the MDF globus authentication web site.  One advantage of this system is that it doesn't actually require authentication at all.  You can use `anonymous=True` and several of the MDF datasets will be available.  However, a number of them will not, and you will have to authenticate using the web to access the entirety of MDF.

In [22]:
from matminer.data_retrieval.retrieve_MDF import MDFDataRetrieval

In [23]:
mdf_dr = MDFDataRetrieval(anonymous=True) # Or anonymous=False if you have a globus login

Error: Service 'mdf' requires authentication.


In [24]:
df = mdf_dr.get_dataframe(elements=['Ag', 'Be'], sources=["oqmd"])

In [25]:
df.head()

,mdf.collection,mdf.composition,mdf.elements.0,mdf.elements.1,mdf.elements.2,mdf.ingest_date,mdf.links.landing_page,mdf.links.metadata.globus_endpoint,mdf.links.metadata.http_host,mdf.links.metadata.path,...,oqmd.delta_e.value,oqmd.magnetic_moment.units,oqmd.magnetic_moment.value,oqmd.spacegroup,oqmd.stability.units,oqmd.stability.value,oqmd.total_energy.units,oqmd.total_energy.value,oqmd.volume.units,oqmd.volume.value
0,OQMD,Ag1Be2,Be,Ag,NaN,2017-08-04T14:19:20.124355Z,http://oqmd.org/analysis/calculation/19856,82f1b5c6-6e9b-11e5-ba47-22000b92c6ec,https://data.materialsdatafacility.org,/collections/oqmd/data/home/oqmd/libraries/ics...,...,0.027908,bohr/atom,NaN,227,eV/atom,0.027908,eV/atom,-3.416156,angstrom^3/atom,10.78560
1,OQMD,Ag1Be2,Be,Ag,NaN,2017-08-04T14:20:06.199847Z,http://oqmd.org/analysis/calculation/69761,82f1b5c6-6e9b-11e5-ba47-22000b92c6ec,https://data.materialsdatafacility.org,/collections/oqmd/data/home/oqmd/libraries/ics...,...,NaN,bohr/atom,NaN,227,NaN,NaN,eV/atom,-3.403616,angstrom^3/atom,10.42060
2,OQMD,Ag3Be1,Be,Ag,NaN,2017-08-04T14:23:25.295316Z,http://oqmd.org/analysis/calculation/176171,82f1b5c6-6e9b-11e5-ba47-22000b92c6ec,https://data.materialsdatafacility.org,/collections/oqmd/data/home/oqmd/libraries/pro...,...,0.271525,bohr/atom,NaN,139,eV/atom,0.271525,eV/atom,-2.783607,angstrom^3/atom,14.46660
3,OQMD,Ag1Be12,Be,Ag,NaN,2017-08-04T14:21:08.479763Z,http://oqmd.org/analysis/calculation/106827,82f1b5c6-6e9b-11e5-ba47-22000b92c6ec,https://data.materialsdatafacility.org,/collections/oqmd/data/home/oqmd/libraries/ics...,...,NaN,bohr/atom,NaN,139,NaN,NaN,eV/atom,-3.572193,angstrom^3/atom,8.77839
4,OQMD,Ag1Be3,Be,Ag,NaN,2017-08-04T14:23:43.224728Z,http://oqmd.org/analysis/calculation/182360,82f1b5c6-6e9b-11e5-ba47-22000b92c6ec,https://data.materialsdatafacility.org,/collections/oqmd/data/home/oqmd/libraries/pro...,...,0.201601,bohr/atom,NaN,139,eV/atom,0.201601,eV/atom,-3.320249,angstrom^3/atom,10.17450


In [26]:
print("There are {} entries in the Ag-Be chemical system".format(len(df)))

There are 504 entries in the Ag-Be chemical system
